# Evaluation of RAG Using Ragas

In the following notebook we'll explore how to evaluate RAG pipelines using a powerful open-source tool called "Ragas". This will give us tools to evaluate component-wise metrics, as well as end-to-end metrics about the performance of our RAG pipelines.

In the following notebook we'll complete the following tasks:

- 🤝 Breakout Room Part #1:
  1. Install required libraries
  2. Set Environment Variables
  3. Creating a simple RAG pipeline with [LangChain v0.2.0](https://python.langchain.com/v0.2/docs/versions/v0_2/)
  4. Synthetic Dataset Generation for Evaluation using the [Ragas](https://github.com/explodinggradients/ragas) framework.
  

- 🤝 Breakout Room Part #2:
  1. Evaluating our pipeline with Ragas
  3. Making Adjustments to our RAG Pipeline
  4. Evaluating our Adjusted pipeline against our baseline
  5. Testing OpenAI's Claim

The only way to get started is to get started - so let's grab our dependencies for the day!

> NOTE: Using this notebook as presented will occur a charge of ~$3USD from OpenAI usage. Most of this cost is produced by the Synthetic Data Generation step - if you want to reduce costs, please use the provided commented code to leverage `GPT-3.5-Turbo` as the `critic_llm`!

## Motivation

A claim, made by OpenAI, is that their `text-embedding-3-small` is better (generally) than their `text-embedding-ada-002` model.

Here's some passages from their [blog](https://openai.com/blog/new-embedding-models-and-api-updates) about the `text-embedding-3` release:

> `text-embedding-3-small` is our new highly efficient embedding model and provides a significant upgrade over its predecessor, the `text-embedding-ada-002` model...

> **Stronger performance.** Comparing `text-embedding-ada-002` to `text-embedding-3-small`, the average score on a commonly used benchmark for multi-language retrieval ([MIRACL](https://github.com/project-miracl/miracl)) has increased from 31.4% to 44.0%, while the average score on a commonly used benchmark for English tasks ([MTEB](https://github.com/embeddings-benchmark/mteb)) has increased from 61.0% to 62.3%.

Well, with a library like Ragas - we can put that claim to the test!

If what they claim is true - we should see an increase on related metrics by using the new embedding model!

# 🤝 Breakout Room Part #1

## Task 1: Installing Required Libraries

A reminder that one of the [key features](https://blog.langchain.dev/langchain-v0-1-0/) of LangChain v0.1.0 is the compartmentalization of the various LangChain ecosystem packages!

So let's begin grabbing all of our LangChain related packages!

In [2]:
!pip install -U -q langchain langchain-openai langchain_core langchain-community langchainhub openai

We'll also get the "star of the show" today, which is Ragas!

In [3]:
!pip install -qU ragas

We'll be leveraging [QDrant](https://qdrant.tech/) again as our LangChain `VectorStore`.

We'll also install `pymupdf` and its dependencies which will allow us to load PDFs using the `PyMuPDFLoader` in the `langchain-community` package!

In [4]:
!pip install -qU qdrant-client pymupdf pandas

## Task 2: Set Environment Variables

Let's set up our OpenAI API key so we can leverage their API later on.

In [5]:
import os
import openai
from getpass import getpass

openai.api_key = getpass("Please provide your OpenAI Key: ")
os.environ["OPENAI_API_KEY"] = openai.api_key

## Task 3: Creating a Simple RAG Pipeline with LangChain v0.1.0

Building on what we learned last week, we'll be leveraging LangChain v0.1.0 and LCEL to build a simple RAG pipeline that we can baseline with Ragas.

## Building our RAG pipeline

Let's review the basic steps of RAG again:

- Create an Index
- Use retrieval to obtain pieces of context from our Index that are similar to our query
- Use a LLM to generate responses based on the retrieved context

Let's get started by creating our index.

> NOTE: We're going to start leaning on the term "index" to refer to our `VectorStore`, `VectorDatabase`, etc. We can think of "index" as the catch-all term, whereas `VectorStore` and the like relate to the specific technologies used to create, store, and interact with the index.

### Creating an Index

You'll notice that the largest changes (outside of some import changes) are that our old favourite chains are back to being bundled in an easily usable abstraction.

We can still create custom chains using LCEL - but we can also be more confident that our pre-packaged chains are creating using LCEL under the hood.

#### Loading Data

Let's start by loading some data!

> NOTE: You'll notice that we're using a document loader from the community package of LangChain. This is part of the v0.2.0 changes that make the base (`langchain-core`) package remain lightweight while still providing access to some of the more powerful community integrations.

In [8]:
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader(
    "https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf",
)

documents = loader.load()

In [9]:
documents[0].metadata

{'source': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf',
 'file_path': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf',
 'page': 0,
 'total_pages': 195,
 'format': 'PDF 1.3',
 'title': 'The Pmarca Blog Archives',
 'author': '',
 'subject': '',
 'keywords': '',
 'creator': '',
 'producer': 'Mac OS X 10.10 Quartz PDFContext',
 'creationDate': "D:20150110020418Z00'00'",
 'modDate': "D:20150110020418Z00'00'",
 'trapped': ''}

#### Transforming Data

Now that we've got our single document - let's split it into smaller pieces so we can more effectively leverage it with our retrieval chain!

We'll start with the classic: `RecursiveCharacterTextSplitter`.

In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 200,
    chunk_overlap = 50
)

documents = text_splitter.split_documents(documents)

Let's confirm we've split our document.

In [11]:
len(documents)

1864

#### Loading OpenAI Embeddings Model

We'll need a process by which we can convert our text into vectors that allow us to compare to our query vector.

Let's use OpenAI's `text-embedding-ada-002` for this task!

In [12]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002"
)

#### Creating a QDrant VectorStore

Now that we have documents - we'll need a place to store them alongside their embeddings.

In [13]:
from langchain_community.vectorstores import Qdrant

qdrant_vector_store = Qdrant.from_documents(
    documents,
    embeddings,
    location=":memory:",
    collection_name="PMarca Blogs",
)

####❓ Question #1:

List out a few of the techniques that Qdrant uses that make it performant.

> NOTE: Check the [documentation](https://qdrant.tech/documentation/overview/) for more information about QDrant!

Vector databases are optimized for storing and querying these high-dimensional vectors efficiently, and they often using specialized data structures and indexing techniques such as Hierarchical Navigable Small World (HNSW) – which is used to implement Approximate Nearest Neighbors – and Product Quantization, among others.

#### Creating a Retriever

To complete our index, all that's left to do is expose our vectorstore as a retriever - which we can do the same way we would in previous version of LangChain!

In [14]:
retriever = qdrant_vector_store.as_retriever()

#### Testing our Retriever

Now that we've gone through the trouble of creating our retriever - let's see it in action!

In [15]:
retrieved_documents = retriever.invoke("What is a rule of thumb for selecting an industry to invest in?")

In [16]:
for doc in retrieved_documents:
  print(doc)

page_content='the existing order — and make sure that those forces of change\nhave a reasonable chance at succeeding.\nSecond rule of thumb:\nOnce you have picked an industry, get right to the center of it' metadata={'source': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf', 'file_path': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf', 'page': 125, 'total_pages': 195, 'format': 'PDF 1.3', 'title': 'The Pmarca Blog Archives', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'Mac OS X 10.10 Quartz PDFContext', 'creationDate': "D:20150110020418Z00'00'", 'modDate': "D:20150110020418Z00'00'", 'trapped': '', '_id': 'eadeea8b59334809802d720cd145fbad', '_collection_name': 'PMarca Blogs'}
page_content='Third rule:\nIn a rapidly changing Held like technology, the best place to\nget experience when you’re starting out is in younger, high-\ngrowth companies.' metadata={'source': 'https://d1lamhf6l6yk6d

### Creating a RAG Chain

Now that we have the "R" in RAG taken care of - let's look at creating the "AG"!

#### Creating a Prompt Template

There are a few different ways we could create our prompt template - we could create a custom template, as seen in the code below, or we could simply pull a prompt from the prompt hub! Let's look at an example of that!

In [17]:
from langchain import hub

retrieval_qa_prompt = hub.pull("langchain-ai/retrieval-qa-chat")

In [18]:
print(retrieval_qa_prompt.messages[0].prompt.template)

Answer any use questions based solely on the context below:

<context>
{context}
</context>


As you can see - the prompt template is simple (and has a small error) - so we'll create our own to be a bit more specific!

In [19]:
from langchain.prompts import ChatPromptTemplate

template = """Answer the question based only on the following context. If you cannot answer the question with the context, please respond with 'I don't know':

Context:
{context}

Question:
{question}
"""

prompt = ChatPromptTemplate.from_template(template)

#### Setting Up our Basic QA Chain

Now we can instantiate our basic RAG chain!

We'll use LCEL directly just to see an example of it - but you could just as easily use an abstraction here to achieve the same goal!

We'll also ensure to pass-through our context - which is critical for RAGAS.

In [22]:
from operator import itemgetter

from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

primary_qa_llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

retrieval_augmented_qa_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": prompt | primary_qa_llm, "context": itemgetter("context")}
)

####🏗️ Activity #1:

Describe the pipeline shown above in simple terms. You can include a diagram if desired.

![image](https://i.imgur.com/Ad31AhL.png)

Let's test it out!

In [23]:
question = "What is a rule of thumb for selecting an industry to invest in?"

result = retrieval_augmented_qa_chain.invoke({"question" : question})

print(result["response"].content)

Get right to the center of it.


In [24]:
question = "What did Pink Floyd have to say about how to proceed when investing in a new industry?"

result = retrieval_augmented_qa_chain.invoke({"question" : question})

print(result["response"].content)
print(result["context"])

I don't know.
[Document(page_content='ask if you can call them again if things change.\nTrust me — they’d much rather be saying “yes” than “no” —\nthey need all the good investments they can get.\nSecond, consider the environment.', metadata={'source': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf', 'file_path': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf', 'page': 15, 'total_pages': 195, 'format': 'PDF 1.3', 'title': 'The Pmarca Blog Archives', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'Mac OS X 10.10 Quartz PDFContext', 'creationDate': "D:20150110020418Z00'00'", 'modDate': "D:20150110020418Z00'00'", 'trapped': '', '_id': '596b6b04c04b468f9176fef4f3120e3a', '_collection_name': 'PMarca Blogs'}), Document(page_content='watching carefully — if everyone agrees right up front that\nwhatever you are doing makes total sense, it probably isn’t a new\nand radical enough idea to justify a

We can already see that there are some improvements we could make here.

For now, let's switch gears to RAGAS to see how we can leverage that tool to provide us insight into how our pipeline is performing!

## Task 4: Synthetic Dataset Generation for Evaluation using Ragas

Ragas is a powerful library that lets us evaluate our RAG pipeline by collecting input/output/context triplets and obtaining metrics relating to a number of different aspects of our RAG pipeline.

We'll be evaluating on every core metric today, but in order to do that - we'll need to create a test set. Luckily for us, Ragas can do that directly!

### Synthetic Test Set Generation

We can leverage Ragas' [`Synthetic Test Data generation`](https://docs.ragas.io/en/stable/concepts/testset_generation.html) functionality to generate our own synthetic QC pairs - as well as a synthetic ground truth - quite easily!

In [25]:
loader = PyMuPDFLoader(
    "https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf",
)

eval_documents = loader.load()

text_splitter_eval = RecursiveCharacterTextSplitter(
    chunk_size = 600,
    chunk_overlap = 50
)

eval_documents = text_splitter_eval.split_documents(eval_documents)

####❓ Question #2:

Why is it important to split our documents using different parameters when creating our synthetic data?

synthetic generation questions answers it give you more real world data scenarios.

In [26]:
len(eval_documents)

624


> NOTE: 🛑 Using this notebook as presented will occur a charge of ~$3USD from OpenAI usage. Most of this cost is produced by the Synthetic Data Generation step - if you want to reduce costs, please use the provided commented code to leverage GPT-3.5-Turbo as the critic_llm. If you're attempting to create a lot of samples please be aware of cost, as well as rate limits. 🛑

In [27]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

generator_llm = ChatOpenAI(model="gpt-3.5-turbo-16k")
# critic_llm = ChatOpenAI(model="gpt-3.5-turbo") <--- If you don't have GPT-4 access, or to reduce cost/rate limiting issues.
critic_llm = ChatOpenAI(model="gpt-4o")
embeddings = OpenAIEmbeddings()

generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    embeddings
)

distributions = {
    simple: 0.5,
    multi_context: 0.4,
    reasoning: 0.1
}

testset = generator.generate_with_langchain_docs(eval_documents, 20, distributions, is_async = False)
testset.to_pandas()

/Users/sidharthasanyal/dev/AIMakersSpace/LLMLearning/LLMLearning/LLMLearning/.conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Filename and doc_id are the same for all nodes.                     
Generating: 100%|██████████| 20/20 [04:26<00:00, 13.35s/it]


,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What are the challenges of not being able to p...,"[2007, so I can’t commit to that, but give me ...",nan,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
1,What are the common layers of risk for a high-...,[as if it’s an onion. Just like you peel an on...,Here are some of the common layers of risk for...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
2,What are some ways to bootstrap a business bef...,[This obviously raises the issue of how you’re...,"Try to raise angel money, or bootstrap oW of i...",simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
3,How can blogging help an entrepreneur secure f...,[looking for funding to blog — about their sta...,Blogging can help an entrepreneur secure fundi...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
4,What are some factors that can lead to people ...,"[people joining the company, people leaving th...",nan,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
5,What is the role of a primary individual contr...,[from scratch. This is a sharp diWerence from ...,A primary individual contributor in a startup ...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
6,What are some techniques that can help entrepr...,[I want to tell you about my new startup” is a...,nan,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
7,What are some strategies for undergrads to gai...,"[undergrads to do some of the work, and being ...",aggressively pursue internship and co-op progr...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
8,How does the concept of commitment relate to o...,[you can reply to a lot of messages with “I’m ...,The concept of commitment relates to organizin...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
9,What is the description of the space opera in ...,"[pandemics, nuclear terrorist attacks, governm...",Large-scale space opera told through a shiaing...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True


####❓ Question #3:

`{simple: 0.5, reasoning: 0.25, multi_context: 0.25}`

What exactly does this mapping refer to?

The distributions {simple: 0.5, reasoning: 0.25, multi_context: 0.25} represent the weight or importance given to different types of tasks or questions within the Retrieval-Augmented Generation (RAG) assessment framework. Here's what each part means:

Simple (0.5): Simple tasks or questions are given a weight of 50%. This means that half of the assessment focuses on straightforward retrieval tasks where the answer can be directly pulled from a single source without much interpretation or synthesis.

Reasoning (0.25): Reasoning tasks or questions are given a weight of 25%. These tasks require more cognitive effort to derive answers. They might involve logical reasoning, understanding context, or drawing inferences from the information retrieved.

Multi-Context (0.25): Multi-context tasks or questions are also given a weight of 25%. These require synthesizing information from multiple sources to generate a coherent and accurate answer, emphasizing the ability to integrate diverse pieces of information.

In this RAG assessment framework, simple retrieval tasks are prioritized (50%), while more complex reasoning and multi-context integration tasks share the remaining focus equally (25% each). This distribution reflects the emphasis on straightforward information retrieval with a balanced but smaller emphasis on higher-order thinking and synthesis skills.

> NOTE: Check out the Ragas documentation on this generation process [here](https://docs.ragas.io/en/stable/concepts/testset_generation.html).

Let's look at the output and see what we can learn about it!

In [28]:
testset.test_data[0]

DataRow(question='What are the challenges of not being able to pull off the tactic of not keeping a schedule in a structured job environment?', contexts=['2007, so I can’t commit to that, but give me a call on Tuesday at\n2:45 and if I’m available, I’ll meet with you.”\nOr, if it’s important, say, “You know what, let’s meet right now.”\nClearly this only works if you can get away with it. If you have\na structured job, a structured job environment, or you’re a CEO,\nit will be hard to pull oW.\nBut if you can do it, it’s really liberating, and will lead to far\nhigher productivity than almost any other tactic you can try.\nThis idea comes from a wonderful book called A Perfect Mess,\nwhich explains how not keeping a schedule has been key to'], ground_truth='nan', evolution_type='simple', metadata=[{'source': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf', 'file_path': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.p

### Generating Responses with RAG Pipeline

Now that we have some QC pairs, and some ground truths, let's evaluate our RAG pipeline using Ragas.

The process is, again, quite straightforward - thanks to Ragas and LangChain!

Let's start by extracting our questions and ground truths from our create testset.

We can start by converting our test dataset into a Pandas DataFrame.

In [29]:
test_df = testset.to_pandas()

In [30]:
test_df

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What are the challenges of not being able to p...,"[2007, so I can’t commit to that, but give me ...",nan,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
1,What are the common layers of risk for a high-...,[as if it’s an onion. Just like you peel an on...,Here are some of the common layers of risk for...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
2,What are some ways to bootstrap a business bef...,[This obviously raises the issue of how you’re...,"Try to raise angel money, or bootstrap oW of i...",simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
3,How can blogging help an entrepreneur secure f...,[looking for funding to blog — about their sta...,Blogging can help an entrepreneur secure fundi...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
4,What are some factors that can lead to people ...,"[people joining the company, people leaving th...",nan,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
5,What is the role of a primary individual contr...,[from scratch. This is a sharp diWerence from ...,A primary individual contributor in a startup ...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
6,What are some techniques that can help entrepr...,[I want to tell you about my new startup” is a...,nan,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
7,What are some strategies for undergrads to gai...,"[undergrads to do some of the work, and being ...",aggressively pursue internship and co-op progr...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
8,How does the concept of commitment relate to o...,[you can reply to a lot of messages with “I’m ...,The concept of commitment relates to organizin...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
9,What is the description of the space opera in ...,"[pandemics, nuclear terrorist attacks, governm...",Large-scale space opera told through a shiaing...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True


In [31]:
test_questions = test_df["question"].values.tolist()
test_groundtruths = test_df["ground_truth"].values.tolist()

Now we'll generate responses using our RAG pipeline using the questions we've generated - we'll also need to collect our retrieved contexts for each question.

We'll do this in a simple loop to see exactly what's happening!

In [32]:
answers = []
contexts = []

for question in test_questions:
  response = retrieval_augmented_qa_chain.invoke({"question" : question})
  answers.append(response["response"].content)
  contexts.append([context.page_content for context in response["context"]])

Now we can wrap our information in a Hugging Face dataset for use in the Ragas library.

In [34]:
from datasets import Dataset

response_dataset = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

Let's take a peek and see what that looks like!

In [35]:
response_dataset[0]

{'question': 'What are the challenges of not being able to pull off the tactic of not keeping a schedule in a structured job environment?',
 'answer': 'It will be hard to pull off not keeping a schedule in a structured job environment because of the structured nature of the job and the job environment.',
 'contexts': ['Don’t keep a schedule\nHe’s crazy, you say!\nI’m totally serious. If you pull it oW — and in many structured\njobs, you simply can’t — this simple tip alone can make a huge\ndiWerence in productivity.',
  'By not keeping a schedule, I mean: refuse to commit to meet-\nings, appointments, or activities at any set time in any future\nday.\nAs a result, you can always work on whatever is most important',
  'Clearly this only works if you can get away with it. If you have\na structured job, a structured job environment, or you’re a CEO,\nit will be hard to pull oW.',
  'The sharpest reaction has been to my theory of not keeping a\nschedule. I’ll stick to my theory but make (o

# 🤝 Breakout Room Part #2

## Task 1: Evaluating our Pipeline with Ragas

Now that we have our response dataset - we can finally get into the "meat" of Ragas - evaluation!

First, we'll import the desired metrics, then we can use them to evaluate our created dataset!

Check out the specific metrics we'll be using in the Ragas documentation:

- [Faithfulness](https://docs.ragas.io/en/stable/concepts/metrics/faithfulness.html)
- [Answer Relevancy](https://docs.ragas.io/en/stable/concepts/metrics/answer_relevance.html)
- [Context Precision](https://docs.ragas.io/en/stable/concepts/metrics/context_precision.html)
- [Context Recall](https://docs.ragas.io/en/stable/concepts/metrics/context_recall.html)
- [Answer Correctness](https://docs.ragas.io/en/stable/concepts/metrics/answer_correctness.html)

See the accompanied presentation for more in-depth explanations about each of the metrics!

In [41]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy
)

metrics = [
    faithfulness,
    answer_relevancy
]

All that's left to do is call "evaluate" and away we go!

In [42]:
results = evaluate(response_dataset, metrics)

Evaluating: 100%|██████████| 40/40 [00:17<00:00,  2.33it/s]


In [43]:
results

{'faithfulness': 0.7769, 'answer_relevancy': 0.8697}

In [44]:
results_df = results.to_pandas()
results_df

,question,answer,contexts,ground_truth,faithfulness,answer_relevancy
0,What are the challenges of not being able to p...,It will be hard to pull off not keeping a sche...,"[Don’t keep a schedule\nHe’s crazy, you say!\n...",nan,1.000000,0.000000
1,What are the common layers of risk for a high-...,"Founder risk, technology risk, and product ris...",[What are the layers of risk for a high-tech\n...,Here are some of the common layers of risk for...,1.000000,0.992970
2,What are some ways to bootstrap a business bef...,Some ways to bootstrap a business before raisi...,[This obviously raises the issue of how you’re...,"Try to raise angel money, or bootstrap oW of i...",0.500000,1.000000
3,How can blogging help an entrepreneur secure f...,Blogging can help an entrepreneur secure fundi...,"[esting things going on, about their point of ...",Blogging can help an entrepreneur secure fundi...,0.750000,1.000000
4,What are some factors that can lead to people ...,"Quality issues, a major customer going bankrup...","[people joining the company, people leaving th...",nan,0.000000,0.846750
5,What is the role of a primary individual contr...,The role of a primary individual contributor i...,[more as administrators and bureaucrats.\n•\nB...,A primary individual contributor in a startup ...,1.000000,0.999999
6,What are some techniques that can help entrepr...,Some techniques that can help entrepreneurs en...,[If\nyou\nengage\nin\na\nset\nof\nthese\ntechn...,nan,1.000000,1.000000
7,What are some strategies for undergrads to gai...,Some strategies for undergrads to gain real-wo...,[What should I do while I’m in school?\nI’m a ...,aggressively pursue internship and co-op progr...,1.000000,1.000000
8,How does the concept of commitment relate to o...,The concept of commitment relates to organizin...,"[As John says, “The list of tasks one has in m...",The concept of commitment relates to organizin...,0.000000,0.995716
9,What is the description of the space opera in ...,"The space opera is described as ""Yat-out huge ...",[space opera told through a shiaing and interl...,Large-scale space opera told through a shiaing...,1.000000,0.954289


## Task 2: Making Adjustments to our RAG Pipeline

Now that we have established a baseline - we can see how any changes impact our pipeline's performance!

Let's modify our retriever and see how that impacts our Ragas metrics!

> NOTE: MultiQueryRetriever is expanded on [here](https://python.langchain.com/docs/modules/data_connection/retrievers/MultiQueryRetriever) but for now, the implementation is not important to our lesson!

In [45]:
from langchain.retrievers import MultiQueryRetriever

advanced_retriever = MultiQueryRetriever.from_llm(retriever=retriever, llm=primary_qa_llm)

We'll also re-create our RAG pipeline using the abstractions that come packaged with LangChain v0.1.0!

First, let's create a chain to "stuff" our documents into our context!

In [46]:
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(primary_qa_llm, retrieval_qa_prompt)

Next, we'll create the retrieval chain!

In [48]:
from langchain.chains import create_retrieval_chain

retrieval_chain = create_retrieval_chain(advanced_retriever, document_chain)

In [49]:
response = retrieval_chain.invoke({"input": "Who is Taylor Swift fueding with?"})

In [51]:
print(response["answer"])

I'm sorry, but based on the context provided, I do not have information about any feud involving Taylor Swift.


In [52]:
response = retrieval_chain.invoke({"input": "Why are they fueding?"})

In [53]:
print(response["answer"])

The text does not provide any information about a feud or conflict between individuals or groups.


Well, just from those responses this chain *feels* better - but lets see how it performs on our eval!

Let's do the same process we did before to collect our pipeline's contexts and answers.

In [54]:
answers = []
contexts = []

for question in test_questions:
  response = retrieval_chain.invoke({"input" : question})
  answers.append(response["answer"])
  contexts.append([context.page_content for context in response["context"]])

Now we can convert this into a dataset, just like we did before.

In [55]:
response_dataset_advanced_retrieval = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

Let's evaluate on the same metrics we did for the first pipeline and see how it does!

In [56]:
advanced_retrieval_results = evaluate(response_dataset_advanced_retrieval, metrics)

Evaluating: 100%|██████████| 40/40 [00:27<00:00,  1.44it/s]


In [57]:
advanced_retrieval_results_df = advanced_retrieval_results.to_pandas()
advanced_retrieval_results_df

,question,answer,contexts,ground_truth,faithfulness,answer_relevancy
0,What are the challenges of not being able to p...,"In a structured job environment, not keeping a...","[By not keeping a schedule, I mean: refuse to ...",nan,0.666667,0.940812
1,What are the common layers of risk for a high-...,Common layers of risk for a high-tech startup ...,[What are the layers of risk for a high-tech\n...,Here are some of the common layers of risk for...,0.800000,0.992970
2,What are some ways to bootstrap a business bef...,Some ways to bootstrap a business before raisi...,[This obviously raises the issue of how you’re...,"Try to raise angel money, or bootstrap oW of i...",0.500000,1.000000
3,How can blogging help an entrepreneur secure f...,Blogging can help an entrepreneur secure fundi...,"[esting things going on, about their point of ...",Blogging can help an entrepreneur secure fundi...,0.833333,1.000000
4,What are some factors that can lead to people ...,Some factors that can lead to people getting f...,"[quality issues, or one of your major customer...",nan,0.000000,1.000000
5,What is the role of a primary individual contr...,A primary individual contributor in a startup ...,[more as administrators and bureaucrats.\n•\nB...,A primary individual contributor in a startup ...,1.000000,0.987285
6,What are some techniques that can help entrepr...,Some techniques that can help entrepreneurs en...,[venture capitalists for follow-on funding. It...,nan,0.583333,0.983983
7,What are some strategies for undergrads to gai...,Some strategies for undergrads to gain real-wo...,[What should I do while I’m in school?\nI’m a ...,aggressively pursue internship and co-op progr...,1.000000,1.000000
8,How does the concept of commitment relate to o...,The concept of commitment relates to organizin...,"[As John says, “The list of tasks one has in m...",The concept of commitment relates to organizin...,0.857143,0.945321
9,What is the description of the space opera in ...,"The space opera is described as ""Yat-out huge ...",[space opera told through a shiaing and interl...,Large-scale space opera told through a shiaing...,0.500000,0.954275


## Task 3: Evaluating our Adjusted Pipeline Against Our Baseline

Now we can compare our results and see what directional changes occured!

Let's refresh with our initial metrics.

In [58]:
results

{'faithfulness': 0.7769, 'answer_relevancy': 0.8697}

And see how our advanced retrieval modified our chain!

In [59]:
advanced_retrieval_results

{'faithfulness': 0.6522, 'answer_relevancy': 0.9616}

In [60]:
import pandas as pd

df_original = pd.DataFrame(list(results.items()), columns=['Metric', 'Baseline'])
df_comparison = pd.DataFrame(list(advanced_retrieval_results.items()), columns=['Metric', 'MultiQueryRetriever with Document Stuffing'])

df_merged = pd.merge(df_original, df_comparison, on='Metric')

df_merged['Delta'] = df_merged['MultiQueryRetriever with Document Stuffing'] - df_merged['Baseline']

df_merged

,Metric,Baseline,MultiQueryRetriever with Document Stuffing,Delta
0,faithfulness,0.776852,0.652202,-0.124649
1,answer_relevancy,0.869679,0.961644,0.091965


## Task 4: Testing OpenAI's Claim

Now that we've seen how our retriever can impact the performance of our RAG pipeline - let's see how changing our embedding model impacts performance.

####🏗️ Activity #2:

Please provide markdown, or code comments, to explain which each of the following steps are doing!

In [ ]:
""" openai embedding model 3 declaration"""
new_embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [ ]:
"""Creating QDrant vector database in memory, populating embeddings from PMarca Blogs embeddings created in the previous step"""
vector_store = Qdrant.from_documents(
    documents,
    new_embeddings,
    location=":memory:",
    collection_name="PMarca Blogs - TE3 - MQR",
)

In [ ]:
#Creating a retreiver for Qdrant vector for vector search 
new_retriever = vector_store.as_retriever()

In [ ]:
#creating  a MultiQueryRetriever from the 
new_advanced_retriever = MultiQueryRetriever.from_llm(retriever=new_retriever, llm=primary_qa_llm)

In [ ]:
#Creating a retreival chain
new_retrieval_chain = create_retrieval_chain(new_advanced_retriever, document_chain)

In [ ]:
#excute langchain and  loops through questions and store the context and answers
answers = []
contexts = []

for question in test_questions:
  response = new_retrieval_chain.invoke({"input" : question})
  answers.append(response["answer"])
  contexts.append([context.page_content for context in response["context"]])

In [ ]:

#creating a dataset with test questions evaluated the answrs context used for ground thuths
new_response_dataset_advanced_retrieval = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

In [ ]:
# dataset evaluation ragas metrics generation
new_advanced_retrieval_results = evaluate(new_response_dataset_advanced_retrieval, metrics)

Evaluating:   0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
# ragas metrics
new_advanced_retrieval_results

{'faithfulness': 0.8658, 'answer_relevancy': 0.9454, 'context_recall': 0.6792, 'context_precision': 0.8472, 'answer_correctness': 0.5055}

In [ ]:
df_baseline = pd.DataFrame(list(results.items()), columns=['Metric', 'ADA + Baseline'])
df_original = pd.DataFrame(list(advanced_retrieval_results.items()), columns=['Metric', 'ADA + MQR'])
df_comparison = pd.DataFrame(list(new_advanced_retrieval_results.items()), columns=['Metric', 'TE3 + MQR'])

df_merged = pd.merge(df_original, df_comparison, on='Metric')
df_merged = pd.merge(df_baseline, df_merged, on="Metric")

df_merged['ADA + MQR -> TE3 + MQR'] = df_merged['TE3 + MQR'] - df_merged['ADA + MQR']
df_merged['Baseline -> TE3 + MQR'] = df_merged['TE3 + MQR'] - df_merged['ADA + Baseline']

df_merged

,Metric,ADA + Baseline,ADA + MQR,TE3 + MQR,ADA + MQR -> TE3 + MQR,Baseline -> TE3 + MQR
0,faithfulness,0.625490,0.712024,0.865833,0.153810,0.240343
1,answer_relevancy,0.852605,0.945499,0.945435,-0.000064,0.092831
2,context_recall,0.650000,0.741667,0.679167,-0.062500,0.029167
3,context_precision,0.758333,0.711185,0.847174,0.135990,0.088841
4,answer_correctness,0.551982,0.552914,0.505525,-0.047388,-0.046457


####❓ Question #4:

Do you think, in your opinion, `text-embedding-3-small` is significantly better than `ada`?

## BONUS ACTIVITY: Using a Better Generator

Now that we've seen how much more effective a better Retrieval pipeline is, let's look at what impact a better(?) Generator is!

Adapt the above `TE3 + MQR` pipeline to use `GPT-4o` and compare the results below!

In [ ]:
### YOUR CODE HERE